## Load required modules

In [8]:
import pandas as pd
import glob 
import utils
import new_utils
import os
import re

## Run exploratory analysis

We are going to find oyt which TFs have atleast 5 motifs and 5 chipseq experiments

In [2]:
# getover5 motifs
taxa_id = 9606
tf_table = "../db-tables/Transcription_Factor-table.csv"
mot_table = "../db-tables/motif-table.tsv"
over5mot = new_utils.getOver5mot(tf_table,mot_table,taxa_id)
over5mot 

/home/festo/Documents/MARStool/MARS-Update-Pipeline/Evaluation/new_utils.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_table['TF_id'] = tf_table.TF_id.astype('str')


,TF_id,TF_name,counts
0,2.3.3.50.1,CTCF [11],109
1,2.1.3.2.1,HNF-4A (NR2A1),71
2,3.5.2.1.1,c-Ets-1,67
3,3.5.2.5.1,PU.1 (Spi-1),60
4,2.1.3.1.1,RXRA (NR2B1),59
...,...,...,...
714,2.3.3.0.100,ZNF35 [11],5
715,3.3.1.4.1,"FOXD1 (Fkh-L8, FREAC4)",5
716,2.3.3.0.97,ZNF547 [10],5
717,1.2.5.5.2,SOHLH2 (TEB1),5


In [3]:
# getover 5 chipseq expts
chipseq = "../hg-chipseq.tsv"
chipdata = "../chipdata_table.tsv"
over5chip = new_utils.getover5chip(tf_table,chipseq,chipdata,taxa_id)
over5chip

,TF_id,TF_name,chip_id,counts
71,2.3.3.50.1,CTCF [11],617,536
37,2.1.1.1.1,Glucocorticoid receptor (GR) (NR3C1),1039,80
2,1.1.1.1.1,c-Jun,863,79
17,1.1.8.1.2,CEBPB (NF-IL6-1),581,58
12,1.1.2.1.4,Fra-2,705,48
...,...,...,...,...
70,2.3.3.37.2,"ZNF384 (CAGH1, CIZ, NMP4, TNRC1) [8]",1500,5
73,2.3.3.74.2,ZNF823 (ZFP36) [16],1414,5
106,3.5.1.5.2,SMARCA5 (SNF2H),552,5
104,3.5.1.1.3,B-Myb (Myb-L2),968,5


In [4]:
# Merge the two to get an overlap
over5chip.rename(columns={'counts': 'chip_counts'},inplace = True)
over5mot.rename(columns={'counts': 'mot_counts'},inplace = True)
over5chip = over5chip[['TF_id','TF_name','chip_counts']]
overlap = pd.merge(over5mot,over5chip,how = 'inner', left_on = ['TF_id', 'TF_name'], 
                   right_on = ['TF_id','TF_name'])
overlap

,TF_id,TF_name,mot_counts,chip_counts
0,2.3.3.50.1,CTCF [11],109,536
1,2.1.3.2.1,HNF-4A (NR2A1),71,5
2,3.5.2.1.1,c-Ets-1,67,9
3,3.5.2.5.1,PU.1 (Spi-1),60,13
4,2.1.3.1.1,RXRA (NR2B1),59,11
...,...,...,...,...
109,3.1.4.4.2,PBX-2,6,5
110,1.2.6.7.6,Mnt,5,14
111,2.3.1.2.9,KLF9 (BTEB1),5,5
112,3.3.1.11.2,FOXK2,5,10


In [5]:
# get a tf list to work on
tf_list = overlap['TF_name'].values
tf_list = list(tf_list)
#tf_list

In [6]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except os.error as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

## Start analysis
We will run tomtom and centrimo

In [13]:
import run_tomtom
import run_centrimo
for tf in tf_list:
    tf_out = re.sub("[\(\[].*?[\)\]]", "", tf).strip()
    tf_out = tf_out.split("(")[0].strip().replace(" ","_")
    
    # get mot list
    motlist, tf_id = new_utils.getmotlist(tf,tf_table,mot_table,taxa_id)
    results = "Results/human/"+ tf_out + "/"
    meme_in = "Data/all_hg_motifs.meme"
    out_meme = results + tf_out + ".meme"
    mkdir_p(results)
    utils.extract_meme_list(meme_in, out_meme, motlist)
    
    # Run Tomtom
    run_tomtom.run_tomtom(tf_out, out_meme, results, figure=True)
    
    # Run centrimo
    prefix = "../"
    chiplist = new_utils.getchiplist(tf_id,chipseq,chipdata,prefix)
    run_centrimo.run_centrimo(tf_out,chiplist,out_meme,results,figure=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Results/human/CTCF//CTCF/tmp/ENCFF519CXF/centrimo.tsv'